In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

input_ = pd.read_excel('./a.integrate.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_[['JOIN_MPN', 'Manufacturer', 'Part_Number']]
input_.drop_duplicates(ignore_index=True)

length = len(input_)

print('总数量：' + str(length))
print()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

import numpy as np

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(length):
    work.put_nowait(i)

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        i = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for _ in range(101):
            try:
                url = 'https://www.rockauto.com/en/partsearch/?mfr=' + input_.loc[i, 'Manufacturer'] + '&partnum=' + input_.loc[i, 'Part_Number']

                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_part = html.xpath('//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]')

                # = = = = = = = = = = = = = = =
                
                list_manufacturer = [part.xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "listing-final-manufacturer")]')[0].text.strip() for part in list_part]

                # = = = = = = = = = = = = = = =
                
                manufacturer_index = np.where(np.array(list_manufacturer) == input_.loc[i, 'Manufacturer'])[0]

                # = = = = = = = = = = = = = = =
                
                list_part_number = [part.xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@title, "Buyer\'s Guide")]')[0].text.strip() for part in list_part]
                
                # = = = = = = = = = = = = = = =
                
                part_number_index = np.where(np.array(list_part_number) == input_.loc[i, 'Part_Number'])[0]
                
                # = = = = = = = = = = = = = = =
                
                part_index = list(np.intersect1d(manufacturer_index, part_number_index))
                
                # = = = = = = = = = = = = = = =
                
                if len(part_index) == 1:
                    part_index = part_index[0]
                    
                    # = = = = = = = = = = = = = = =
                
                    list_note_1 = list_part[part_index].xpath('./descendant::div[contains(@class, "listing-text-row-moreinfo-truck")]/span[contains(@class, "span-link-underline-remover")]/text()')

                    note_1 = '' if len(list_note_1) == 0 else ';'.join([note_1.strip() for note_1 in list_note_1])

                    # = = = = = = = = = = = = = = =
                
                    list_note_2 = list_part[part_index].xpath('./descendant::span[contains(@title, "Please make sure")]/span/text()')

                    note_2 = '' if len(list_note_2) == 0 else ';'.join([note_2.strip() for note_2 in list_note_2])

                    # = = = = = = = = = = = = = = =
                    
                    src = list_part[part_index].xpath('./descendant::img[contains(@id, "inlineimg_thumb")]')

                    src = '' if len(src) == 0 else 'https://www.rockauto.com' + src[0].xpath('./@src')[0].strip().replace('__ra_m.', '.')

                    # = = = = = = = = = = = = = = =
                    
                    oe = list_part[part_index].xpath('./descendant::span[contains(@title, "Replaces these Alternate")]')

                    oe = '' if len(oe) == 0 else oe[0].xpath('./text()')[0].strip().replace(', ', ';')

                    # = = = = = = = = = = = = = = =
                    
                    part_json = json.loads(list_part[part_index].xpath('./descendant::input[contains(@id, "listing_data_essential")]/@value')[0])

                    part_type = part_json['parttype'].strip()
                    part_key = part_json['partkey'].strip()

                    # = = = = = = = = = = = = = = =
                
                    crawler_status = 'ok'

                    break
                
            except:
                time.sleep(0.3)
                continue

        if crawler_status == 'ok':
            df_temp = pd.DataFrame([{'JOIN_MPN': input_.loc[i, 'JOIN_MPN'],
                                     'Vehcile': '',
                                     'OE': oe,
                                     'Note_1': note_1,
                                     'Note_2': note_2,
                                     'Pic': '',
                                     'Url': url,
                                     'Info': 'https://www.rockauto.com/en/moreinfo.php?pt=' + part_type + '&pk=' + part_key,
                                     'Src': src,
                                     'Part_Type': part_type,
                                     'Part_Key': part_key}])

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
        else:
            df_temp = pd.DataFrame([{'JOIN_MPN': input_.loc[i, 'JOIN_MPN'],
                                     'Manufacturer': input_.loc[i, 'Manufacturer'],
                                     'Part_Number': input_.loc[i, 'Part_Number']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        print(input_.loc[i, 'Manufacturer'] + ' - ' + input_.loc[i, 'Part_Number'] + '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['JOIN_MPN'], ascending=[True])
output_correct.to_excel('./b.part_number-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['JOIN_MPN'], ascending=[True])
    output_error.to_excel('./part_number_error.xlsx', index=False)
    print()
    print('爬虫存在error')
print()
print('爬虫结束')

总数量：24601

ROCKAUTO - T10  <->  [ok] - 剩余数量：24581 - 当前时间：13:44:13
ROCKAUTO - GIFTCERT75  <->  [ok] - 剩余数量：24580 - 当前时间：13:44:15


KeyboardInterrupt
2024-02-16T05:44:16Z


KeyboardInterrupt: 